<a href="https://colab.research.google.com/github/geoaigroup/models-deployment/blob/main/scripts/process_large_tiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import argparse
import os
import cv2

from segmentation_models_pytorch import Unet

from skimage.io import imread,imsave
import numpy as np
import rasterio as rio
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio

from utils import *
# from color_map import cm_data
from rasterio.features import shapes
from tqdm.notebook import tqdm

/home/jamada/jupyterlab/eo-xai/xai_fyp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = '/home/jamada/UrbanModels/weights_pytorch/tu-tf_efficientnet_b0_Unet_whu_3-classes_40-epochs_TRY001'
THRESH = 0.5
ALPHA = 0.85
SCALE = None

#load your model with pre-trained weights
model = Unet(
        encoder_name = "tu-tf_efficientnet_b0",
        encoder_depth= 5,
        encoder_weights = None,
        decoder_use_batchnorm = True,
        decoder_channels = (256, 128, 64, 32, 16),
        decoder_attention_type = None,
        in_channels= 3,
        classes = 3,
        activation = 'sigmoid',
        aux_params = None,
    )

model = load_model(model,MODEL_PATH)
model.cuda()

Unet(
  (encoder): TimmUniversalEncoder(
    (model): EfficientNetFeatures(
      (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn1): BatchNormAct2d(
              32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid(

In [7]:
in_dir = r'/home/jamada/jupyterlab/eo-xai/WHUNotebook/'
out_dir = r'/home/jamada/jupyterlab/eo-xai/whu_noise_training/results'


directory = os.fsencode(in_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):         
        raster_file = rio.open(f'{in_dir}/{filename}')
        print("raster", raster_file)
        full_img = raster_file.read().transpose(1,2,0)
        full_img,rrp_info = ratio_resize_pad(full_img, ratio = None)
        print(full_img.shape)
        if full_img.shape[-1]==4:full_img = cv2.cvtColor(full_img,cv2.COLOR_RGBA2RGB) #WHU images are RGBA
        print("full image",full_img.shape)
        full_img = normalize(full_img)
        x = totensor(full_img)
        x = x.cuda()

        with torch.no_grad():
            print("input shape",x.shape)
            y_pred = model(x)
            
            print("sssssss", y_pred.shape)
            y_pred = unpad_resize(y_pred,rrp_info)

        mask = y_pred[0,...].cpu().numpy().transpose(1,2,0)
        print('predicted mask shape: ', mask.shape)


        ##post_process
        thresh = 0.5
        instances = post_process(mask,thresh = thresh,thresh_b = 0.5,mina=100,mina_b=50)  
        instances[instances>0] =1
        print("instances shape", instances.shape)


        #to png
        os.makedirs(out_dir,exist_ok=True)

        imageio.imwrite(f'{out_dir}/results_{filename}', mask[:,:,1])


        


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


raster <open DatasetReader name='/home/jamada/jupyterlab/eo-xai/WHUNotebook//whu_example1.png' mode='r'>
(512, 512, 4)
full image (512, 512, 3)
input shape torch.Size([1, 3, 512, 512])
sssssss torch.Size([1, 3, 512, 512])
predicted mask shape:  (512, 512, 3)
instances shape (512, 512)
raster <open DatasetReader name='/home/jamada/jupyterlab/eo-xai/WHUNotebook//whu_example2.png' mode='r'>
(512, 512, 4)
full image (512, 512, 3)
input shape torch.Size([1, 3, 512, 512])
sssssss torch.Size([1, 3, 512, 512])
predicted mask shape:  (512, 512, 3)


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


instances shape (512, 512)
raster <open DatasetReader name='/home/jamada/jupyterlab/eo-xai/WHUNotebook//whu_example3.png' mode='r'>
(512, 512, 4)
full image (512, 512, 3)
input shape torch.Size([1, 3, 512, 512])
sssssss torch.Size([1, 3, 512, 512])
predicted mask shape:  (512, 512, 3)
instances shape (512, 512)
